In [2]:
import os
import cv2
import yaml
import torch
from torch import nn
import numpy as np
import random
from torchvision import transforms

from models import ScreenVAE
from experiment import VAEXperiment

import sys
sys.path.append('../procedural-advml')
from utils_noise import PlotGaborAni

In [3]:
config_file = "configs/my_vae.yaml"
encode_path = os.path.join(r"logs\Screentone-VAE\version_54", r"checkpoints\epoch=47-step=258863.ckpt")
decode_path = os.path.join(r"logs\ParamDecoder", r"epoch-32.ckpt")
global_path = r"D:\shared\datasets\20220214\gabor_noise_band_5_10"

In [4]:
# in -> in 8 層，in -> out 8 層
class MLP(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(MLP, self).__init__()
        modules = []
        for _ in range(8):
            modules.append(
                nn.Sequential(nn.Linear(in_channel, in_channel))
            )
        modules.append(
            nn.Sequential(nn.Linear(in_channel, out_channel))
        )
        self.fc = nn.Sequential(*modules)
        self.loss_mse = torch.nn.MSELoss(reduction='sum')

    def forward(self, x):
        return self.fc(x)
    
    def loss(self, real, fake):
        return self.loss_mse(real, fake)

In [5]:
def param(x):
    p = {}
    p['num_kern'] = int(x[0] * 200)
    p['ksize'] = int(x[1] * 40)
    p['sigma'] = float(x[2] * 20)
    p['theta'] = float(x[3] * 3.14)
    p['lambd'] = float(x[4] * 10)
    p['sides'] = int(x[5] * 12)
    return p

In [6]:

# 解析路徑中的參數至 tensor
def parse_path(path):
    X = np.zeros((len(path), 6))
    axis0_idx = 0
    for p in path:
        p = os.path.basename(p)[:-4]
        x = [float(s) for s in p.split("_")]
        x[0] /= 200
        x[1] /= 40
        x[2] /= 20
        x[3] /= 3.14
        x[4] /= 10
        x[5] /= 12
        X[axis0_idx] = np.array(x)
        axis0_idx += 1
    return torch.FloatTensor(X)

In [7]:
with open(config_file, 'r', encoding="utf-8") as file:
    try:
        config = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print("error", exc)
output_path = f"./{config['logging_params']['save_dir']}ParamDecoder"

In [8]:
encoder = ScreenVAE(**config['model_params'])
print("Loading encoder")
encoder.load_state_dict(torch.load(encode_path))
print("Loading encoder ... Done")


Loading encoder
Loading encoder ... Done


In [9]:
# 準備圖片
# print("Preparing Dataset")
# experiment = VAEXperiment(encoder, config['exp_params'], select_rect=True)
# dataloader = experiment.predict_dataloader()
# print("Preparing Dataset ... Done")

In [10]:
# create decoder
decoder = MLP(config['model_params']["latent_dim"], 6)
print("Loading decoder")
decoder.load_state_dict(torch.load(decode_path))
print("Loading decoder ... Done")

Loading decoder
Loading decoder ... Done


In [11]:
# 產生一張隨機圖片
def getRandomImg(size):
    s_num_kern = random.randint(5, 200)
    s_ksize = random.randint(3, 40)
    s_sigma = random.uniform(2, 20)
    s_theta = random.uniform(0, np.pi)
    s_lambd = random.uniform(5, 10)
    s_sides = random.randint(1, 12)
    img = PlotGaborAni(s_num_kern, s_ksize, s_sigma, s_theta, s_lambd, s_sides, size=size)
    x = [s_num_kern / 200, s_ksize / 40, s_sigma / 20, s_theta / 3.14, s_lambd / 10, s_sides / 12]
    return img, x

In [12]:
# 產生 n 張隨機圖片
def getRandomImgs(n, size):
    image = []
    X = []
    for i in range(n):
        img, x = getRandomImg(size)
        X.append(x)
        image.append(img)
    return image, X

In [13]:
# 生成結果圖
# def output(real, fake):
#     size = len(real)
#     img_size = 128
#     margin = 5
#     next_distance = img_size + margin
#     imgs = np.zeros((size * next_distance - margin, 2 * img_size + margin), int)
#     for i in range(size):
#         try:
#             real_img = PlotGaborAni(**param(real[i, :]))
#             fake_img = PlotGaborAni(**param(fake[i, :]))
#             startH = i * next_distance
#             imgs[startH:startH + img_size, :img_size] = real_img
#             imgs[startH:startH + img_size, next_distance:] = fake_img
#         except:
#             print(f"error {fake[i, :]}, {real[i, :]}")
#     cv2.imwrite(f"{output_path}/re-generate.png", imgs)

In [14]:
# 生成結果圖 (list)
def output_list(X):
    w_size, h_size = len(X), len(X[0])
    img_size = 512
    margin = 5
    next_distance = img_size + margin
    imgs = np.zeros((h_size * next_distance - margin, w_size * next_distance - margin), int)
    for i in range(w_size):
        x = X[i]
        for j in range(h_size):
            try:
                startH = j * next_distance
                startW = i * next_distance
                imgs[startH:startH + img_size, startW:startW + img_size] = PlotGaborAni(size=img_size, **param(x[j, :]))
            except:
                print(f"error {x[j, :]}")
    cv2.imwrite(f"{output_path}/re-generate.png", imgs)

In [15]:

# # 自動生成並辨識參數重建
# # get real parameter
# imgs, real = getRandomImgs(5)
# # get fake parameter
# mu, log_var = encoder.encode(imgs.float())
# z = encoder.reparameterize(mu, log_var)
# fake = decoder(z)

# loss = decoder.loss(real, fake)
# fake = fake.cpu().detach().numpy()
# real = real.cpu().detach().numpy()

# output_list([real, fake])

In [16]:
# 自動生成並辨識參數重建多次
# get real parameter
real_img_size = 1024
imgs, real = getRandomImgs(5, real_img_size)
# get fake parameter

real = torch.FloatTensor(real)
X = [real.cpu().detach().numpy()]
img_size = config['exp_params']['img_size']
for _ in range(10):
    print(_)
    def RandomCrop(ori_size, crop_size):
        def crop(img):
            h, w = random.randrange(0, ori_size - crop_size), random.randrange(0, ori_size - crop_size)
            return img[h:h+crop_size, w:w+crop_size]
        return crop
    tmp_imgs = [transforms.ToTensor()(RandomCrop(real_img_size, img_size)(img)) for img in imgs]
    tmp_imgs = torch.stack(tmp_imgs, 0)
    
    mu, log_var = encoder.encode(tmp_imgs.float())
    z = encoder.reparameterize(mu, log_var)
    fake = decoder(z)

    loss = decoder.loss(real, fake)
    X.append(fake.cpu().detach().numpy())

# output_list([X[0], X[1], X[-1]])
output_list(X)

0
1
2
3
4
5
6
7
8
9
